In [74]:
import os
import keyring
import glob
from ftplib import FTP
from arcgis import GIS
# from arcgis.features import GeoAccessor
# from arcgis.features import GeoSeriesAccessor
# from arcgis.features import FeatureLayerCollection
# from arcgis.features import FeatureLayer


# Toggles

In [ ]:
# Switch to false to disable
update_maps = False
update_charts = False

# Update files in arcgis online

In [2]:
# Sign into ArcGIS Online
un = 'analytics_wfrc'
pw = keyring.get_password('AGSServicesStatus', 'Analytics AGOL')
gis = GIS(username=un, password=pw)

In [36]:
# store path to the new data
cityarea_zip = r".\Outputs\map\CITYAREA.gdb.zip"
county_zip = r".\Outputs\map\COUNTY.gdb.zip"
distmed_zip = r".\Outputs\map\DISTMED.gdb.zip"
otherarea_zip = r".\Outputs\map\OTHERAREA.gdb.zip"
taz_zip = r".\Outputs\map\TAZ.gdb.zip"

In [37]:
# store item ids
cityarea_item_id = '265bfbe2cae7490196e230eced9109b2'
county_item_id = 'cb97017428fe470280fd015b4d7d61bf'
distmed_item_id = 'bdf485e1e5e94c16b08b2464bc4f50a5'
otherarea_item_id = '56ae4b1483e54eff932c24127d496f46'
taz_item_id = '844f78208e604aabb0d14d5ae827712d'

In [ ]:
# get the item using the id
cityarea_item = gis.content.get(cityarea_item_id)
county_item = gis.content.get(county_item_id)
distmed_item = gis.content.get(distmed_item_id)
otherarea_item = gis.content.get(otherarea_item_id)
taz_item = gis.content.get(taz_item_id)

In [ ]:
if update_maps == True:
    
    # update the data source, then republish
    cityarea_item.update(data=cityarea_zip)
    cityarea_item.publish(overwrite=True)

    county_item.update(data=county_zip)
    county_item.publish(overwrite=True)

    distmed_item.update(data=distmed_zip)
    distmed_item.publish(overwrite=True)

    otherarea_item.update(data=otherarea_zip)
    otherarea_item.publish(overwrite=True)

    taz_item.update(data=taz_zip)
    taz_item.publish(overwrite=True)

In [3]:
# ## Access the feature layer as a GIS content item
# item_id = 'adf71d1f547b4a5581afc41328deeebd'
# item = gis.content.get(item_id)
# # feature_layer = item.layers[0]

# v1 = r"E:\Projects\REMM-Process-SE-Output-For-Web\Outputs\C_TEST_1.gdb.zip"
# v2 = r"E:\Projects\REMM-Process-SE-Output-For-Web\Outputs\New\C_TEST_1.gdb.zip"

# item.update(data=v2)
# item.publish(overwrite=True)

# Update files on server

In [76]:
local_chart_folder = r'.\Outputs\chart'
un2= 'wfrcorg'
pw2 = keyring.get_password('AGSServicesStatus', 'WFRC File Server')
ip = keyring.get_password('AGSServicesStatus', 'WFRC File Server IP')
ftp = FTP(ip)
ftp.login(user=un2, passwd =pw2)

'230 OK. Current restricted directory is /'

In [ ]:
if update_charts == True:
    geogs = ['CITYAREA', 'COUNTY', 'DISTMED', 'OTHERAREA', 'TAZ']

    for geog in geogs:
        ftp.cwd(f'/public_html/household-job-forecast-map/widgets/SESidebar/data/chart/{geog}')
        
        old_jsons = [item for item in ftp.nlst() if item not in ['.','..'] and item.endswith('.json')]
        print('deleting old jsons...')
        for json in old_jsons:
            print(json)
            ftp.delete(json)

        print('uploading new jsons...')
        new_jsons = glob.glob(os.path.join(local_chart_folder, geog, '*.json'))
        for json in new_jsons:
            with open(json,'rb') as file:
                ftp.storlines(os.path.basename(json), file)

In [ ]:
# close the connection
ftp.quit()